# Neural network for segmentation of welds




### From Mario Bartolvic, Hamza Mani and Viet Phuoc Ho

### Requirements

In [ ]:
! pip install git+https://github.com/divamgupta/image-segmentation-keras tqdm opencv-python

### Imports

In [ ]:
import cv2
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from IPython.display import Image
from keras_segmentation.models.unet import unet
from keras_segmentation.models.segnet import segnet
from keras_segmentation.models.fcn import fcn_32

### Initialize the model

In [ ]:
model = unet(n_classes=2)
#model = segnet(n_classes=2)
#model = fcn_32(n_classes=2)


### Image preprocessing

In [ ]:
for file in os.listdir("train/masks/"): # replace paths
  img = cv2.imread(os.path.join("train/masks", file), cv2.IMREAD_GRAYSCALE)
  normalizedImg = np.zeros((800, 800))
  normalizedImg = cv2.normalize(img,  normalizedImg, 0, 1, cv2.NORM_MINMAX)
  
  if img is not None:
    cv2.imwrite(os.path.join("train/masks", file), normalizedImg)

### Train the model

In [ ]:
model.train(
    train_images =  "train/images/",
    train_annotations = "train/masks/",
    checkpoints_path = "/weights/unet_1" , 
    epochs=10,
    steps_per_epoch=500,
    val_steps_per_epoch=500,
)

### Evaluation of the model 

In [ ]:
for index in tqdm(range(0, 19)):
  out = model.predict_segmentation(
      inp=f"test/images/{index}.png",
      out_fname=f"out/{index}_out.png"
  )

### Result visualisation 

In [ ]:
img_out = cv2.imread("test/2_out.png", 0)
img_mask = cv2.imread("test/2_real.png", 0)
mask = np.zeros((800, 800))
out = cv2.normalize(img_out,  mask, 0, 1, cv2.NORM_MINMAX)
real = cv2.normalize(img_mask,  mask, 0, 1, cv2.NORM_MINMAX)

In [ ]:
result = out.copy()
true_pos = 0
false_pos = 0
for outer_index, outer in enumerate(real):
  for inner_index, inner in enumerate(outer):
    if inner == 1: # needs to be background
      if out[outer_index][inner_index] == 1: # prediction correct
        result[outer_index][inner_index] = 255
      elif out[outer_index][inner_index] == 0: # detected to much
        result[outer_index][inner_index] = 150
    elif inner == 0: # should be prediction 
      if out[outer_index][inner_index] == 0: # prediction correct
        result[outer_index][inner_index] = 0
        true_pos += 1
      elif out[outer_index][inner_index] == 1: # not detected
        result[outer_index][inner_index] = 75
        false_pos += 1

In [ ]:
prediction = true_pos/(true_pos + false_pos)
prediction

In [ ]:
lab = cv2.cvtColor(result, cv2.COLOR_GRAY2RGB)

correct = lab[100][100].copy()
correct[0] = 255
correct[1] = 0
correct[2] = 0

much = lab[100][100].copy() # detected to much
much[0] = 0
much[1] = 255
much[2] = 0

less = lab[100][100].copy() # not detected
less[0] = 0
less[1] = 0
less[2] = 255

for outer_index, outer in enumerate(result):
  for inner_index, inner in enumerate(outer):
    if 0 in lab[outer_index][inner_index]:
      lab[outer_index][inner_index] = correct
    elif 150 in lab[outer_index][inner_index]:
      lab[outer_index][inner_index] = much
    elif 75 in lab[outer_index][inner_index]:
      lab[outer_index][inner_index] = less

img_mask = cv2.imwrite("test/result.png", lab)